In [66]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
#     del data['shop_score_service']
#     del data['shop_score_description']
#     del data['shop_review_positive_rate']
#     del data['shop_score_delivery']
    return data

In [67]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False)

train


## 特征选择

In [68]:
dataa.columns

Index(['instance_id', 'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'property_2_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'property_0_pvnum', 'property_1_pvnum',
       'property_2_pvnum', 'item_id_pvnum', 'occupation_click',
       'user_gender_click'],
      dtype='object')

In [69]:
tezheng = ['instance_id','item_id_pvnum', 'occupation_click','category_1_pvnum', 'user_click']

In [70]:
dataa = dataa[tezheng]

In [71]:

train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [72]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 35)
2018-09-24 23:59:47
(357066, 35)
(121021, 35)


In [73]:

print('test')
test_a = pd.read_csv('test1.csv',index_col=False)
data2a = pd.read_csv('test1_all.csv',index_col=False) 

# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test


## 特征选择

In [74]:
data2a = data2a[tezheng]

In [75]:
test_a.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [76]:
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
# del train['context_id']
# del val['context_id']
# del test_a['context_id']
# del train['item_brand_id']
# del val['item_brand_id']
# del test_a['item_brand_id']
# del train['category_0']
# del val['category_0']
# del test_a['category_0']
# del train['category_2']
# del val['category_2']
# del test_a['category_2']
# del train['property_1']
# del val['property_1']
# del test_a['property_1']
# del train['property_2']
# del val['property_2']
# del test_a['property_2']

(42888, 30)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [77]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [78]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:11
2018-09-23 00:00:11
1 days 23:59:33
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(42888, 33)


In [79]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
# del train['item_id']
# del test_a['item_id']
# del val['item_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_id', 'item_brand_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'item_id_pvnum', 'occupation_click',
       'category_1_pvnum', 'user_click', 'category_0', 'category_1',
       'category_2', 'property_0', 'property_1', 'property_2', 'halfhour',
       'user_count', 'item_count'],
      dtype='object')
(520975, 590)


In [80]:
# LogisticRegression?

In [81]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [82]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [83]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=50,
#                         max_depth = 8,
                        
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 2
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.683895
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674839
[3]	valid_0's binary_logloss: 0.665953
[4]	valid_0's binary_logloss: 0.657234
[5]	valid_0's binary_logloss: 0.648689
[6]	valid_0's binary_logloss: 0.640307
[7]	valid_0's binary_logloss: 0.63208
[8]	valid_0's binary_logloss: 0.624015
[9]	valid_0's binary_logloss: 0.616094
[10]	valid_0's binary_logloss: 0.608317
[11]	valid_0's binary_logloss: 0.600687
[12]	valid_0's binary_logloss: 0.593199
[13]	valid_0's binary_logloss: 0.585842
[14]	valid_0's binary_logloss: 0.578621
[15]	valid_0's binary_logloss: 0.571521
[16]	valid_0's binary_logloss: 0.564546
[17]	valid_0's binary_logloss: 0.557695
[18]	valid_0's binary_logloss: 0.550965
[19]	valid_0's binary_logloss: 0.544357
[20]	valid_0's binary_logloss: 0.53786
[21]	valid_0's binary_logloss: 0.531476
[22]	valid_0's binary_logloss: 0.52519
[23]	valid_0's binary_logloss: 0.519018
[24]	valid_0's binary_logloss

[207]	valid_0's binary_logloss: 0.123431
[208]	valid_0's binary_logloss: 0.122908
[209]	valid_0's binary_logloss: 0.12239
[210]	valid_0's binary_logloss: 0.121878
[211]	valid_0's binary_logloss: 0.12137
[212]	valid_0's binary_logloss: 0.12087
[213]	valid_0's binary_logloss: 0.120375
[214]	valid_0's binary_logloss: 0.119887
[215]	valid_0's binary_logloss: 0.119402
[216]	valid_0's binary_logloss: 0.118929
[217]	valid_0's binary_logloss: 0.118462
[218]	valid_0's binary_logloss: 0.117996
[219]	valid_0's binary_logloss: 0.11754
[220]	valid_0's binary_logloss: 0.117092
[221]	valid_0's binary_logloss: 0.116645
[222]	valid_0's binary_logloss: 0.116201
[223]	valid_0's binary_logloss: 0.115766
[224]	valid_0's binary_logloss: 0.115336
[225]	valid_0's binary_logloss: 0.114914
[226]	valid_0's binary_logloss: 0.114496
[227]	valid_0's binary_logloss: 0.114082
[228]	valid_0's binary_logloss: 0.113672
[229]	valid_0's binary_logloss: 0.113267
[230]	valid_0's binary_logloss: 0.112868
[231]	valid_0's bina

[405]	valid_0's binary_logloss: 0.0848684
[406]	valid_0's binary_logloss: 0.0848289
[407]	valid_0's binary_logloss: 0.0847887
[408]	valid_0's binary_logloss: 0.0847458
[409]	valid_0's binary_logloss: 0.0847073
[410]	valid_0's binary_logloss: 0.0846629
[411]	valid_0's binary_logloss: 0.0846229
[412]	valid_0's binary_logloss: 0.084583
[413]	valid_0's binary_logloss: 0.0845443
[414]	valid_0's binary_logloss: 0.0845004
[415]	valid_0's binary_logloss: 0.0844581
[416]	valid_0's binary_logloss: 0.0844143
[417]	valid_0's binary_logloss: 0.0843806
[418]	valid_0's binary_logloss: 0.0843397
[419]	valid_0's binary_logloss: 0.0843087
[420]	valid_0's binary_logloss: 0.084272
[421]	valid_0's binary_logloss: 0.0842346
[422]	valid_0's binary_logloss: 0.0842007
[423]	valid_0's binary_logloss: 0.0841639
[424]	valid_0's binary_logloss: 0.0841255
[425]	valid_0's binary_logloss: 0.0840961
[426]	valid_0's binary_logloss: 0.0840579
[427]	valid_0's binary_logloss: 0.0840237
[428]	valid_0's binary_logloss: 0.08

[606]	valid_0's binary_logloss: 0.0813055
[607]	valid_0's binary_logloss: 0.0813
[608]	valid_0's binary_logloss: 0.0812906
[609]	valid_0's binary_logloss: 0.0812835
[610]	valid_0's binary_logloss: 0.0812755
[611]	valid_0's binary_logloss: 0.0812693
[612]	valid_0's binary_logloss: 0.0812648
[613]	valid_0's binary_logloss: 0.0812587
[614]	valid_0's binary_logloss: 0.0812511
[615]	valid_0's binary_logloss: 0.0812468
[616]	valid_0's binary_logloss: 0.0812429
[617]	valid_0's binary_logloss: 0.0812346
[618]	valid_0's binary_logloss: 0.0812292
[619]	valid_0's binary_logloss: 0.0812228
[620]	valid_0's binary_logloss: 0.0812154
[621]	valid_0's binary_logloss: 0.081207
[622]	valid_0's binary_logloss: 0.081201
[623]	valid_0's binary_logloss: 0.081197
[624]	valid_0's binary_logloss: 0.0811928
[625]	valid_0's binary_logloss: 0.0811844
[626]	valid_0's binary_logloss: 0.0811784
[627]	valid_0's binary_logloss: 0.0811723
[628]	valid_0's binary_logloss: 0.0811658
[629]	valid_0's binary_logloss: 0.081161

[804]	valid_0's binary_logloss: 0.0806318
[805]	valid_0's binary_logloss: 0.0806285
[806]	valid_0's binary_logloss: 0.0806271
[807]	valid_0's binary_logloss: 0.0806256
[808]	valid_0's binary_logloss: 0.080624
[809]	valid_0's binary_logloss: 0.0806238
[810]	valid_0's binary_logloss: 0.0806213
[811]	valid_0's binary_logloss: 0.0806192
[812]	valid_0's binary_logloss: 0.0806183
[813]	valid_0's binary_logloss: 0.0806168
[814]	valid_0's binary_logloss: 0.0806158
[815]	valid_0's binary_logloss: 0.0806147
[816]	valid_0's binary_logloss: 0.0806144
[817]	valid_0's binary_logloss: 0.0806146
[818]	valid_0's binary_logloss: 0.0806124
[819]	valid_0's binary_logloss: 0.0806125
[820]	valid_0's binary_logloss: 0.0806104
[821]	valid_0's binary_logloss: 0.0806092
[822]	valid_0's binary_logloss: 0.0806099
[823]	valid_0's binary_logloss: 0.0806071
[824]	valid_0's binary_logloss: 0.0806064
[825]	valid_0's binary_logloss: 0.080604
[826]	valid_0's binary_logloss: 0.0806019
[827]	valid_0's binary_logloss: 0.08

[1006]	valid_0's binary_logloss: 0.0804522
[1007]	valid_0's binary_logloss: 0.080452
[1008]	valid_0's binary_logloss: 0.0804528
[1009]	valid_0's binary_logloss: 0.0804539
[1010]	valid_0's binary_logloss: 0.0804546
[1011]	valid_0's binary_logloss: 0.080454
[1012]	valid_0's binary_logloss: 0.0804543
[1013]	valid_0's binary_logloss: 0.0804559
[1014]	valid_0's binary_logloss: 0.080456
[1015]	valid_0's binary_logloss: 0.0804572
[1016]	valid_0's binary_logloss: 0.0804543
[1017]	valid_0's binary_logloss: 0.0804537
[1018]	valid_0's binary_logloss: 0.0804537
[1019]	valid_0's binary_logloss: 0.0804533
[1020]	valid_0's binary_logloss: 0.0804523
[1021]	valid_0's binary_logloss: 0.0804521
[1022]	valid_0's binary_logloss: 0.0804519
[1023]	valid_0's binary_logloss: 0.0804511
[1024]	valid_0's binary_logloss: 0.0804514
[1025]	valid_0's binary_logloss: 0.0804499
[1026]	valid_0's binary_logloss: 0.0804484
[1027]	valid_0's binary_logloss: 0.0804466
[1028]	valid_0's binary_logloss: 0.0804444
[1029]	valid_0

[1199]	valid_0's binary_logloss: 0.0803795
[1200]	valid_0's binary_logloss: 0.0803786
[1201]	valid_0's binary_logloss: 0.0803783
[1202]	valid_0's binary_logloss: 0.0803795
[1203]	valid_0's binary_logloss: 0.0803799
[1204]	valid_0's binary_logloss: 0.0803791
[1205]	valid_0's binary_logloss: 0.0803777
[1206]	valid_0's binary_logloss: 0.0803776
[1207]	valid_0's binary_logloss: 0.0803782
[1208]	valid_0's binary_logloss: 0.0803772
[1209]	valid_0's binary_logloss: 0.0803773
[1210]	valid_0's binary_logloss: 0.0803752
[1211]	valid_0's binary_logloss: 0.0803733
[1212]	valid_0's binary_logloss: 0.0803734
[1213]	valid_0's binary_logloss: 0.0803728
[1214]	valid_0's binary_logloss: 0.0803714
[1215]	valid_0's binary_logloss: 0.0803707
[1216]	valid_0's binary_logloss: 0.08037
[1217]	valid_0's binary_logloss: 0.0803681
[1218]	valid_0's binary_logloss: 0.0803691
[1219]	valid_0's binary_logloss: 0.0803697
[1220]	valid_0's binary_logloss: 0.0803687
[1221]	valid_0's binary_logloss: 0.0803682
[1222]	valid_

[1392]	valid_0's binary_logloss: 0.080328
[1393]	valid_0's binary_logloss: 0.0803286
[1394]	valid_0's binary_logloss: 0.0803275
[1395]	valid_0's binary_logloss: 0.0803261
[1396]	valid_0's binary_logloss: 0.0803262
[1397]	valid_0's binary_logloss: 0.0803259
[1398]	valid_0's binary_logloss: 0.0803266
[1399]	valid_0's binary_logloss: 0.0803269
[1400]	valid_0's binary_logloss: 0.0803278
[1401]	valid_0's binary_logloss: 0.0803285
[1402]	valid_0's binary_logloss: 0.0803287
[1403]	valid_0's binary_logloss: 0.0803285
[1404]	valid_0's binary_logloss: 0.0803276
[1405]	valid_0's binary_logloss: 0.0803287
[1406]	valid_0's binary_logloss: 0.0803274
[1407]	valid_0's binary_logloss: 0.0803267
[1408]	valid_0's binary_logloss: 0.0803268
[1409]	valid_0's binary_logloss: 0.0803266
[1410]	valid_0's binary_logloss: 0.080327
[1411]	valid_0's binary_logloss: 0.080327
[1412]	valid_0's binary_logloss: 0.0803271
[1413]	valid_0's binary_logloss: 0.0803276
[1414]	valid_0's binary_logloss: 0.08033
[1415]	valid_0's

[1585]	valid_0's binary_logloss: 0.0803395
[1586]	valid_0's binary_logloss: 0.0803403
[1587]	valid_0's binary_logloss: 0.0803414
[1588]	valid_0's binary_logloss: 0.0803418
[1589]	valid_0's binary_logloss: 0.0803423
[1590]	valid_0's binary_logloss: 0.0803399
[1591]	valid_0's binary_logloss: 0.0803402
[1592]	valid_0's binary_logloss: 0.0803383
[1593]	valid_0's binary_logloss: 0.0803379
[1594]	valid_0's binary_logloss: 0.0803384
[1595]	valid_0's binary_logloss: 0.0803392
[1596]	valid_0's binary_logloss: 0.0803401
[1597]	valid_0's binary_logloss: 0.0803417
[1598]	valid_0's binary_logloss: 0.0803421
[1599]	valid_0's binary_logloss: 0.0803421
[1600]	valid_0's binary_logloss: 0.0803422
[1601]	valid_0's binary_logloss: 0.0803424
[1602]	valid_0's binary_logloss: 0.0803423
Early stopping, best iteration is:
[1452]	valid_0's binary_logloss: 0.0803246


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
       reg_alpha=2, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [84]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0816995451384

Start predicting...
0.077626447734
0.0803246127504


In [63]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [64]:
y_sub_1.shape

(42888,)

In [65]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180417.txt',sep=" ",index=False)